In [9]:
import sys
import math
import copy
import pandas as pd
import numpy as np
import heat_transfer as bht
import ht

import model_ht as mht
import model_temperatures as mtemp

from CoolProp.CoolProp import PropsSI

import model_transfers as mtr

import sympy as sp

import model_fins as modfins

from latex2sympy2 import latex2sympy, latex2latex


In [10]:
bc = 'free_end'

In [11]:
x, L_fin, lambd, Ac, k, Bi, T_ext, T_0 = sp.symbols('x L_fin a Ac k Bi T_ext T_0')

# Define the temperature profile solution
A = sp.Symbol('A')
B = sp.Symbol('B')

# Temperature function T(x)
T = T_ext + A * sp.cosh(sp.sqrt(2 * Bi) * x/lambd) + B * sp.sinh(sp.sqrt(2 * Bi) * x/lambd)

# Boundary condition at x = 0 (T(0) = T_0)
boundary_condition_0 = sp.Eq(T.subs(x, 0), T_0)

# Solve for A
A_solution = sp.solve(boundary_condition_0, A)[0]

# Substitute the value of A in T(x)
T = T.subs(A, A_solution)

dTdx = sp.diff(T, x)

if bc == 'free_end':
    boundary_condition_L_fin = sp.Eq( dTdx.subs(x, L_fin), -Bi/lambd * (T.subs(x, L_fin) - T_ext) )
    B_solution = sp.solve(boundary_condition_L_fin, B)[0]
elif bc == 'adiabatic':
    boundary_condition_L_fin = sp.Eq( dTdx.subs(x, L_fin), 0)
    B_solution = sp.solve(boundary_condition_L_fin, B)[0]   
else:
    raise ValueError('Boundary condition not recognized')

T = T.subs(B, B_solution)
dTdx = dTdx.subs(B, B_solution)

expression = (T-T_ext) / (T_0-T_ext)
expression = sp.simplify(expression)

if bc == "adiabatic":
    expression = sp.simplify(expression * sp.cosh(sp.sqrt(2 * Bi) * L_fin/lambd)) / sp.cosh(sp.sqrt(2 * Bi) * L_fin/lambd)

gamma = sp.diff(expression,x).subs(x, 0)
gamma = -k * Ac * sp.simplify(gamma)

Qdot =  gamma * (T_0 - T_ext)

In [13]:
B_solution

-(sqrt(2)*sqrt(Bi)*T_0*sinh(sqrt(2)*sqrt(Bi)*L_fin/a) - sqrt(2)*sqrt(Bi)*T_ext*sinh(sqrt(2)*sqrt(Bi)*L_fin/a) + Bi*T_0*cosh(sqrt(2)*sqrt(Bi)*L_fin/a) - Bi*T_ext*cosh(sqrt(2)*sqrt(Bi)*L_fin/a))/(sqrt(2)*sqrt(Bi)*cosh(sqrt(2)*sqrt(Bi)*L_fin/a) + Bi*sinh(sqrt(2)*sqrt(Bi)*L_fin/a))

In [16]:
Bp = latex2sympy(r"-\frac{\left( T_0 - T_{ext} \right) \left( \sqrt{\frac{Bi}{2}} \cosh{\left( \frac{\sqrt{2 Bi} L_{fin}}{a}\right)} + \sinh{\left( \frac{\sqrt{2 Bi} L_{fin}}{a} \right)} \right)}{\cosh{\left( \frac{\sqrt{2 Bi} L_{fin}}{a} \right)} + \sqrt{\frac{Bi}{2}} \sinh{\left( \frac{\sqrt{2 Bi} L_{fin}}{a} \right)}}")

In [17]:
Bp

-(T_0 - T_{ext})*(sqrt((B*i)/2)*cosh((L_{fin}*sqrt(2*B*i))/a) + sinh((L_{fin}*sqrt(2*B*i))/a))/(sqrt((B*i)/2)*sinh((L_{fin}*sqrt(2*B*i))/a) + cosh((L_{fin}*sqrt(2*B*i))/a))

In [ ]:
gamma

In [20]:
latex_eq = sp.latex(gamma)

print(latex_eq)

\frac{Ac k \left(\sqrt{2} Bi^{\frac{3}{2}} \cosh{\left(\frac{\sqrt{2} \sqrt{Bi} L_{fin}}{a} \right)} + 2 Bi \sinh{\left(\frac{\sqrt{2} \sqrt{Bi} L_{fin}}{a} \right)}\right)}{a \left(\sqrt{2} \sqrt{Bi} \cosh{\left(\frac{\sqrt{2} \sqrt{Bi} L_{fin}}{a} \right)} + Bi \sinh{\left(\frac{\sqrt{2} \sqrt{Bi} L_{fin}}{a} \right)}\right)}
